In [51]:
library(Seurat)
library(ggplot2)
library(umap)
library(glue)

In [58]:
expression_matrix = read.csv("/Users/shayecarver/condPCA/final_method/test_matrix.txt", sep="\t", row.names=1 )
seurat_object <- CreateSeuratObject(counts = t(expression_matrix) )
metadata_df = read.csv("/Users/shayecarver/condPCA/final_method/test_metadata.txt", sep="\t", row.names=1)
seurat_object <- AddMetaData(object = seurat_object, metadata = metadata_df)
seurat_object = NormalizeData(seurat_object, normalization.method = "LogNormalize", scale.factor = 10000)
seurat_object = FindVariableFeatures(seurat_object, nfeatures = 2000)
seurat_object <- ScaleData(seurat_object, features = seurat_object@assays$RNA@var.features, scale.max = 999999999999)


Centering and scaling data matrix



In [59]:
# standardize data in base R: batch, age, sex

standardized = t(as.matrix(seurat_object@assays$RNA@scale.data) )

for ( i in 1:dim(standardized)[2] ) {
    standardized[,i] = resid(lm(standardized[,i] ~ as.factor(seurat_object@meta.data$Batch) + scale(seurat_object@meta.data$Age) + as.factor(
seurat_object@meta.data$Sex) ))
}

#write.table(standardized, "/Users/shayecarver/condPCA/final_method/base_R_standardized_residual_test_matrix.txt", sep = "\t")


In [61]:
dim(standardized)

[1] 5000 2000

In [65]:
# perform PCA on standardized data

pca = prcomp( standardized , center = FALSE, scale = FALSE )
dim(pca$rotation[,1:50])
eigenvectors <- pca$rotation[,1:50]
# Extract eigenvalues
eigenvalues <- pca$sdev^2


write.table(eigenvectors,"/Users/shayecarver/condPCA/final_method/base_R_PCA_STAND.txt", sep = "\t")
write.table(eigenvalues,"/Users/shayecarver/condPCA/final_method/base_R_PCA_STAND_eigenvalues.txt", sep = "\t")

# df.pca_r = data.frame(
#     "PC1" = pca$rotation[,1] , 
#     "PC2" = pca$rotation[,2] )

# # perform umap on df.pca_r
# umap_output = umap(df.pca_r)

# # Extract the first two dimensions from the UMAP output
# umap_embeddings <- umap_output$layout[, 1:2]

# # Create a new data frame with UMAP embeddings
# umap_df <- data.frame(UMAP1 = umap_embeddings[, 1], UMAP2 = umap_embeddings[, 2])

# # Plot the first two UMAP embeddings
# ggplot(umap_df, aes(x = UMAP1, y = UMAP2)) +
#   geom_point() +
#   labs(title = "UMAP: First Two Embeddings")

#   # plot the initial two PCs of PCA
# ggplot(df.pca_r, aes(x=PC1, y=PC2 )) + geom_point(alpha=0.5) + theme_minimal()






[1] 2000   50

In [66]:
standardized = t(as.matrix(seurat_object@assays$RNA@scale.data) )

for ( i in 1:dim(standardized)[2] ) {
    standardized[,i] = resid(lm(standardized[,i] ~ as.factor(seurat_object@meta.data$celltype) + as.factor(seurat_object@meta.data$Batch) + scale(seurat_object@meta.data$Age) + as.factor(
seurat_object@meta.data$Sex) ))
}
#write.table(standardized, "/Users/shayecarver/condPCA/final_method/base_R_standardized_residual_test_matrix_w_CELLTYPE.txt", sep = "\t")
pca = prcomp( standardized , center = FALSE, scale = FALSE )

eigenvectors <- pca$rotation[,1:50]
# Extract eigenvalues
eigenvalues <- pca$sdev^2


write.table(eigenvectors,"/Users/shayecarver/condPCA/final_method/base_R_PCA_COND.txt", sep = "\t")
write.table(eigenvalues,"/Users/shayecarver/condPCA/final_method/base_R_PCA_COND_eigenvalues.txt", sep = "\t")
# df.pca_r = data.frame(
#     "PC1" = pca$rotation[,1] , 
#     "PC2" = pca$rotation[,2] )

# ggplot(df.pca_r, aes(x=PC1, y=PC2 )) + geom_point(alpha=0.5) + theme_minimal()

# # perform umap on df.pca_r
# umap_output = umap(df.pca_r)

# # Extract the first two dimensions from the UMAP output
# umap_embeddings <- umap_output$layout[, 1:2]

# # Create a new data frame with UMAP embeddings
# umap_df <- data.frame(UMAP1 = umap_embeddings[, 1], UMAP2 = umap_embeddings[, 2])

# # Plot the first two UMAP embeddings
# ggplot(umap_df, aes(x = UMAP1, y = UMAP2)) +
#   geom_point() +
#   labs(title = "UMAP: First Two Embeddings")


In [56]:
# for ( i in 1:length(celltypes) ) {
CT = celltypes[1]
print(CT)

expression_matrix = read.csv("/Users/shayecarver/condPCA/final_method/test_matrix.txt", sep="\t", row.names=1 )
seurat_object <- CreateSeuratObject(counts = t(expression_matrix) )
metadata_df = read.csv("/Users/shayecarver/condPCA/final_method/test_metadata.txt", sep="\t", row.names=1)
seurat_object <- AddMetaData(object = seurat_object, metadata = metadata_df)

celltypes = unique(seurat_object@meta.data$celltype)

my_method <- read.table("/Users/shayecarver/condPCA/final_method/output.txt", sep = "\t", header = TRUE, row.names = 1)
vargenes = colnames(my_method)


seurat_object_subset = subset(seurat_object, subset = celltype == CT)

seurat_object_subset = NormalizeData(seurat_object_subset, normalization.method = "LogNormalize", scale.factor = 10000)
#seurat_object_subset = FindVariableFeatures(seurat_object_subset, nfeatures = 500)
#seurat_object_subset <- ScaleData(seurat_object_subset, features = seurat_object_subset@assays$RNA@var.features, scale.max = 999999999999)
seurat_object_subset <- ScaleData(seurat_object_subset, features = vargenes, scale.max = 999999999999)

sub_metadata = seurat_object@meta.data[seurat_object@meta.data$celltype == "ODC",]

standardized = t(as.matrix(seurat_object_subset@assays$RNA@scale.data) )

for ( i in 1:dim(standardized)[2] ) {
    standardized[,i] = resid(lm(standardized[,i] ~ as.factor(sub_metadata$Batch) + scale(sub_metadata$Age) + as.factor(
sub_metadata$Sex) ))
}

write.table(standardized, glue("/Users/shayecarver/condPCA/final_method/base_R_standardized_residual_iter_PCA_{CT}_SCANPY_VARGENES_NOT_SEURAT.txt"), sep = "\t")

[1] "ODC"


Centering and scaling data matrix

